In [ ]:
# this piece of code converts the raw csv file to processed input
import os
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd

from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

porter = PorterStemmer()

data = pd.read_csv('earbud_reviews.csv', encoding='utf-8')
#stop_words = set(stopwords.words('english'))

a = 0
LL = {} # dict to store num of appearance
punctuation_string = string.punctuation # store the punctuation

# the following piece computes the number of appearance of each word in the whole text and store in a dict
for line in data.values:
    stri = str(line[13]) # this is the review body
    stri = stri.replace("  ", " ") # to eliminate some double space appear together
    L = nltk.word_tokenize(stri) # tokenlize 
    for word in L:
        if word.isalnum(): # only keep the words that contain only letters and numbers
            stemmed = porter.stem(word.lower()) # stem the word
            if stemmed in LL:
                LL[stemmed] += 1 # if the word in alr in dict, add the number of appearance
            else:
                LL[stemmed] = 0 # else number of appearance is 0
            #LL.append(porter.stem(word)) # append each word of each review body to LL
    a += 1 
    print(a) # just to show the progress

# store the number of appearance into a file for further use
with open('temp.txt','a+', encoding='utf-8') as f:
    for key in LL.keys():
        f.write(str(key) + " " + str(LL[key]) + "\n")
    f.close()


# store the 
with open('earbud.txt','a+', encoding='utf-8') as f: # store the data in this file
    with open('new_vote.txt','a+', encoding = 'utf-8') as ff:
        with open('o.txt', 'a+', encoding = 'utf-8') as fff:
            temp = 0 
            for line in data.values:
                stri = str(line[13]) # the original review 
                L = nltk.word_tokenize(stri)
                LLL = [word.lower() for word in L if word.isalnum()]
                if len(LLL) > 1: # if the processed review has more than 1 word inside
                    filtered_words = []
                    for word in LLL:
                        stemmed = porter.stem(word)
                        if (stemmed not in stopwords.words('english')) and (stemmed in LL) and (LL[stemmed] >= 10):
                        # only keep the words that are not stopwords in english and appears more than 10 times in the whole corpus
                            filtered_words.append(stemmed)
                    new = " ".join(filtered_words)

                    t = str(line[1]) + "_" + str(line[2]) + " " + str(line[7]) + " " + new
                    # line[1] is the customer id
                    # line [2] is the review id
                    # line[7] is the actual rating
                    # t should look like this:
                    # 13000908_R27F4OF4BIA4LU 2 last long stop work within year

                    processed_content = t.split(" ", 2)[2] 

                    if processed_content != "" and processed_content != " " and processed_content != "\t": # if processed_content actually contains comething
                        f.write(t + '\n')
                        fff.write(str(line[13])+"\n") # record the original content into a new file
                        ff.write((str(line[8]) + " " + str(int(line[9]) - int(line[8]))+ "\n")) # get the up and down vote and write to new file

                        temp+=1
                        print(temp) # to show the progress
            fff.close()
        ff.close()
    f.close()
            


In [ ]:

# write all the data into csv file

#customer id, review id, actual rating, predicted rating, processed content, original content, up vote, down vote, prob 1-10, entropy, review length

# earbud_reviews.txt: customer_id review_id
import csv  
import nltk
from scipy.stats import entropy
file = open("AIR_document_topic") # output file

LL = []
header = ['Cutomer ID', 'Review ID', 'Actual Rating', 'Predicted Rating', 'Processed Content', 'Original Content', 'Up Vote', 'Down Vote',
    'prob_a1', 'prob_a2', 'prob_a3', 'prob_a4', 'prob_a5', 'prob_a6', 'prob_a7', 'prob_a8', 'prob_a9', 'prob_a10', 'Entropy', 'Review Length']

LL.append(header)
topic_score = file.readlines() # extract data


with open("filtered_tokenlized.txt") as file2:
    info = lines = [line.rstrip('\n') for line in file2]
    # info is all the lines

file3 = open("AIR_score") # predicted score by the algo
f3 = file3.readlines()

file4 = open("o.txt") 
# this is the file containing the otiginal content
f4 = file4.readlines()

file5 = open("new_vote.txt")
# file containing up and down votes
f5 = file5.readlines()

for i in range(630526): # number of reviews after filter
    L = []
    temp = topic_score[i].split(" ")[0].split("\t") # this is each score
    t = info[i] # info for ith line
    tempp = [int(x) for x in temp] # convert str to int
    summation = sum(tempp) # sum of all the numbers 

    processed_content = t.split(" ", 2)[2]

    score = f3[i][:-1].split("\t")
    scoree = [float(x) for x in score][1:]

    predicted_score = sum(scoree)/10

    cus_review_id = t.split(" ", 1)[0].split("_")

    original_content = f4[i].rstrip("\n")

    up_down = f5[i].rstrip("\n").split(" ")
    upvotes = int(up_down[0])
    downvotes = int(up_down[1])

    ent = entropy(list(tempp), base=2) # get the entropy

    words = nltk.word_tokenize(original_content)
    words=[word for word in words if word.isalnum()]


    L.append(cus_review_id[0])
    L.append(cus_review_id[1])
    L.append(score[0])
    L.append(predicted_score)
    L.append(processed_content)
    # original content
    L.append(original_content)
    L.append(upvotes)
    L.append(downvotes)

    for num in tempp:
        # convert the probability to %
        L.append("%.2f" % (num/summation*100) + "%" + "  ")

    L.append(ent) # entropy
    L.append(len(words)) # review length

    LL.append(L)
    print(i) # show the progress

print(len(LL))


with open('new.csv', 'a+', encoding='UTF8') as f:
    writer = csv.writer(f)

    writer.writerows(LL)

f.close()